In [28]:
!aws s3 sync ./data/ s3://llava-ue1/data/

upload: data/piechart-QA.jsonl to s3://llava-ue1/data/piechart-QA.jsonl                
upload: data/piechart-QA/.ipynb_checkpoints/image_2-checkpoint.png to s3://llava-ue1/data/piechart-QA/.ipynb_checkpoints/image_2-checkpoint.png
upload: data/piechart-QA/.ipynb_checkpoints/image_39-checkpoint.png to s3://llava-ue1/data/piechart-QA/.ipynb_checkpoints/image_39-checkpoint.png
upload: data/piechart-QA/.ipynb_checkpoints/image_25-checkpoint.png to s3://llava-ue1/data/piechart-QA/.ipynb_checkpoints/image_25-checkpoint.png
upload: data/piechart-QA/.ipynb_checkpoints/image_10-checkpoint.png to s3://llava-ue1/data/piechart-QA/.ipynb_checkpoints/image_10-checkpoint.png
upload: data/piechart-QA/.ipynb_checkpoints/image_31-checkpoint.png to s3://llava-ue1/data/piechart-QA/.ipynb_checkpoints/image_31-checkpoint.png
upload: data/piechart-QA/.ipynb_checkpoints/image_38-checkpoint.png to s3://llava-ue1/data/piechart-QA/.ipynb_checkpoints/image_38-checkpoint.png
upload: data/piechart-QA/.ipynb_checkp

In [1]:
!git clone https://github.com/haotian-liu/LLaVA.git

Cloning into 'LLaVA'...
remote: Enumerating objects: 2247, done.
remote: Counting objects: 100% (1271/1271), done.
remote: Compressing objects: 100% (532/532), done.
remote: Total 2247 (delta 874), reused 784 (delta 739), pack-reused 976
Receiving objects: 100% (2247/2247), 13.86 MiB | 40.10 MiB/s, done.
Resolving deltas: 100% (1434/1434), done.


In [36]:
%%writefile LLaVA/finetune-lora-piechart-QA.sh

#!/bin/bash
export WANDB_MODE=offline

cd /opt/ml/code
pip install -e . --no-deps

deepspeed llava/train/train_mem.py \
    --lora_enable True --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5 \
    --deepspeed ./scripts/zero3.json \
    --model_name_or_path liuhaotian/llava-v1.5-7b \
    --version v1 \
    --data_path /opt/ml/input/data/piechart/piechart-QA.jsonl \
    --image_folder /opt/ml/input/data/piechart/piechart-QA \
    --vision_tower openai/clip-vit-large-patch14-336 \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir /opt/ml/checkpoints/$(job_id) \
    --num_train_epochs 5 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 8 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb


Overwriting LLaVA/finetune-lora-piechart-QA.sh


In [37]:
# Initialize sagemaker session and get the training data s3 uri
import json
import time
import boto3
import numpy as np
import sagemaker
import sagemaker.huggingface
import os

ROLE = sagemaker.get_execution_role()
sess = sagemaker.Session()
BUCKET = "llava-ue1"
PREFIX = "data"
s3uri = os.path.join("s3://", BUCKET, PREFIX)
print(f"sagemaker role arn: {ROLE}")
print(f"sagemaker bucket: {BUCKET}")
print(f"sagemaker session region: {sess.boto_region_name}")
print(f"data uri: {s3uri}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::348052051973:role/service-role/SageMaker-ExecutionRole-20231205T130265
sagemaker bucket: llava-ue1
sagemaker session region: us-east-1
data uri: s3://llava-ue1/data


In [38]:
from time import gmtime, strftime
job_id = "llava-v15-7b-task-lora-"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(job_id)

llava-v15-7b-task-lora-2024-02-17-07-33-50


In [39]:
environment = {
        'job_id': job_id
}

# Define metrics definitions, such metrics will be extracted from training script's printed logs and send to cloudwatch
metric_definitions=[
        {'Name': 'loss', 'Regex': "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
        {'Name': 'learning_rate', 'Regex': "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
        {'Name': 'epoch', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"},
        {'Name': 'train_runtime', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"},
        {'Name': 'train_samples_per_second', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"},
        {'Name': 'train_steps_per_second', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"},
        {'Name': 'train_loss', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"}
]

In [40]:
# Point the training data to the s3 uri. Use FastFile to "mount" the s3 files directly instead of copying to local disk
from sagemaker.inputs import TrainingInput

training_input = TrainingInput(
    s3_data_type='S3Prefix', # Available Options: S3Prefix | ManifestFile | AugmentedManifestFile
    s3_data=s3uri,
    distribution='FullyReplicated', # Available Options: FullyReplicated | ShardedByS3Key 
    input_mode='FastFile'
)

In [41]:
from sagemaker.huggingface import HuggingFace

instance_type = 'ml.g5.12xlarge'
use_spot_instances = True
max_run=600
max_wait = 1200 if use_spot_instances else None

output_uri = os.path.join("s3://", BUCKET, job_id, "output")
checkpoint_uri = os.path.join("s3://", BUCKET, job_id, "checkpoints")


huggingface_estimator = HuggingFace(entry_point='finetune-lora-piechart-QA.sh',
                                    source_dir='./LLaVA',
                                    instance_type=instance_type,
                                    instance_count=1,
                                    py_version='py310',
                                    image_uri='348052051973.dkr.ecr.us-east-1.amazonaws.com/llava-training:latest',
                                    role=ROLE,
                                    metric_definitions = metric_definitions,
                                    environment=environment,
                                    use_spot_instances=use_spot_instances,
                                    max_run=max_run,
                                    max_wait=max_wait,
                                    output_path=output_uri,
                                    checkpoint_s3_uri=checkpoint_uri,
                                   )

huggingface_estimator.fit({'piechart': training_input}, job_name=job_id)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: llava-v15-7b-task-lora-2024-02-17-07-33-50
